W ramach laboratorium należy zaimplementować algorytmy obliczające spójność krawędziową grafu.

$\color{yellow}{\text{Zadanie 1.}}$
Opracuj i zaimplementuj algorytm obliczający spójność krawędziową zadanego grafu G, wykorzystując algorytm Forda-Fulkersona oraz następujący fakt:

(Tw. Mengera) Minimalna ilość krawędzi które należy usunąć by zadane wierzchołki s, t znalazły się w różnych komponentach spójnych jest równa ilości krawędziowo rozłącznych ścieżek pomiędzy s i t

In [1]:
from dimacs import *
from queue import deque
from queue import PriorityQueue
import os

In [2]:
def readSolution(graph):
    with open(graph, "r") as f:
        first_line = f.readline()
    return int(first_line.split("=")[1])

def compareResults(file_path, graph_name, method, show_results=False):
    """Compares the result from the file with the result from the given method."""
    file_solution = readSolution(file_path)
    algorithm_solution = method(file_path)

    print("Graph name:", graph_name)
    if file_solution == algorithm_solution:
        print("Test passed")
    else:
        print("Test failed")
        show_results = True

    if show_results:
        print("Solution from file: ", file_solution)
        print("Solution from algorithm: ", algorithm_solution)

def testAllGraphs(directory_path, method):
    """Tests all graphs in the given directory with the specified method."""
    for filename in os.listdir(directory_path):
        if filename == 'grid100x100':
            continue

        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):
            compareResults(file_path, filename, method)

Algorytm Forda Fulkersona, znajduje maksymalny przepływ między zadanymi wierzchołkami.

In [3]:
def convertGraph(graph):
    n, graph = loadWeightedGraph(graph)

    G = [[] for _ in range(n)]
    matrix = [[0 for _ in range(n)] for _ in range(n)]


    for (u, v, w) in graph:
        u -= 1
        v -= 1

        G[u].append(v)
        G[v].append(u)
        matrix[u][v] = 1
        matrix[v][u] = 1

    return G, matrix

def findPathBfs(graph, matrix, s, t):
    q = deque()
    q.append(s)

    parent = [None for _ in range(len(graph))]
    parent[s] = s

    while len(q) > 0:
        u = q.popleft()

        for v in graph[u]:
            if matrix[u][v] > 0 and parent[v] == None:
                parent[v] = u
                q.append(v)

    if parent[t] == None:
        return None

    path = []
    u = t
    while u != s:
        path.append(u)
        u = parent[u]

    path.append(s)
    path.reverse()

    return path

def fordFulkerson(graph, s, t):
    graph, matrix = convertGraph(graph)
    n = len(graph)
    flow = 0

    path = findPathBfs(graph, matrix, s, t)
    while path:
        if len(path) == 1:
            path = [0] + path
        
        flow += 1

        for u, v in zip(path, path[1:]):
            matrix[u][v] -= 1
            matrix[v][u] += 1

        path = findPathBfs(graph, matrix, s, t)
    return flow

Wywołanie algorymtu Forda Fulkersona dla każdej pary wierzchołków i zwrócenie najmniejszego z otrzymanych wyników.

In [4]:
def edgeConsitencyFf(graph):
    result = float('inf')

    G, _ = convertGraph(graph)
    n = len(G)
    for u in range(n):
        for v in range(u + 1, n):
            result = min(result, fordFulkerson(graph, u, v))

    return result

In [17]:
print(edgeConsitencyFf("graphs-lab3/geo20_2b"))
print(edgeConsitencyFf("graphs-lab3/mc2"))
print(edgeConsitencyFf("graphs-lab3/rand20_100"))

2
2
2


$\color{yellow}{\text{Zadanie 2.}}$
Proszę zaimplementować program obliczający spójność krawędziową grafu nieskierowanego G przy użyciu algorytmu Stoera-Wagnera.

In [6]:
class Node:
  def __init__(self):
    self.edges = {}    # słownik  mapujący wierzchołki do których są krawędzie na ich wagi

  def addEdge( self, to, weight):
    self.edges[to] = self.edges.get(to,0) + weight  # dodaj krawędź do zadanego wierzchołka
                                                    # o zadanej wadze; a jeśli taka krawędź
                                                    # istnieje, to dodaj do niej wagę

  def delEdge( self, to ):
    del self.edges[to]                              # usuń krawędź do zadanego wierzchołka

In [7]:
def buildGraph(G):
    (V, L) = loadWeightedGraph(G)
    graph = [Node() for _ in range(V)]

    for (x, y, c) in L:
      x -= 1
      y -= 1
      graph[x].addEdge(y, c)
      graph[y].addEdge(x, c)

    vertices = set(range(0, V))  # Create a set of vertices
    return graph, vertices

def printGraph(G):
  for v in range(len(G)):
    if G[v].exists:
      print(v, ":", G[v].edges)

In [8]:
def mergeVertices(G, x, y):
    for v, w in G[y].edges.items():
        if v != x:
            G[x].addEdge(v, w)
            G[v].addEdge(x, w)
        G[v].delEdge(y)
    G[y].edges = {}

In [9]:
def minimumCutPhase(G, V):
    Q = PriorityQueue()
    a = 0
    S = set()
    S.add(a)
    n_cut = [0 for _ in range(len(G))]

    for v, weight in G[a].edges.items():
        n_cut[v] = -weight
        Q.put((-weight, v))

    visited = set()
    visited.add(a)
    t = None
    s = a

    while not Q.empty():
        curr_n_cut, v = Q.get()

        if v in visited:
            continue

        visited.add(v)
        cut = -curr_n_cut
        t = s
        s = v

        for u, weight in G[v].edges.items():
            n_cut[u] -= weight
            if u not in visited:
                Q.put((n_cut[u], u))

    mergeVertices(G, s, t)

    return (t, cut)

In [10]:
def edgeConsistencySw(path):
    G, V = buildGraph(path)
    result = float('inf')
    
    while len(V) > 1:
        y, pot_result = minimumCutPhase(G, V)
        V.remove(y)
        result = min(result, pot_result)
    return result

In [11]:
testAllGraphs("graphs-lab3", edgeConsistencySw)

Graph name: clique100
Test passed
Graph name: clique20
Test passed
Graph name: clique200
Test passed
Graph name: clique5
Test passed
Graph name: cycle
Test passed
Graph name: geo100_2a
Test passed
Graph name: geo20_2b
Test passed
Graph name: geo20_2c
Test passed
Graph name: grid5x5
Test passed
Graph name: mc1
Test passed
Graph name: mc2
Test passed
Graph name: path
Test passed
Graph name: rand100_500
Test passed
Graph name: rand20_100
Test passed
Graph name: simple
Test passed
Graph name: trivial
Test passed
